In [219]:
# SK-learn libraries for feature extraction from text.
from sklearn.feature_extraction.text import *
import pandas as pd
import re
import nltk
import json
import numpy as np
from nltk import sent_tokenize, word_tokenize
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from string import punctuation
from nltk.stem import PorterStemmer
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
from collections import Counter
# conda install textblob -c conda-forge
from textblob import TextBlob

In [24]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /Users/aliceye/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/aliceye/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/aliceye/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/aliceye/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [6]:
train_json_data = json.load(open('train.json'))
train_df = pd.json_normalize(train_json_data)

test_json_data = json.load(open('test.json'))
test_df = pd.json_normalize(test_json_data)

In [30]:
test_df.columns

Index(['giver_username_if_known', 'request_id', 'request_text_edit_aware',
       'request_title', 'requester_account_age_in_days_at_request',
       'requester_days_since_first_post_on_raop_at_request',
       'requester_number_of_comments_at_request',
       'requester_number_of_comments_in_raop_at_request',
       'requester_number_of_posts_at_request',
       'requester_number_of_posts_on_raop_at_request',
       'requester_number_of_subreddits_at_request',
       'requester_subreddits_at_request',
       'requester_upvotes_minus_downvotes_at_request',
       'requester_upvotes_plus_downvotes_at_request', 'requester_username',
       'unix_timestamp_of_request', 'unix_timestamp_of_request_utc'],
      dtype='object')

In [182]:
available_cols = ['giver_username_if_known', 'request_id', 'request_text_edit_aware',
       'request_title', 'requester_account_age_in_days_at_request',
       'requester_days_since_first_post_on_raop_at_request',
       'requester_number_of_comments_at_request',
       'requester_number_of_comments_in_raop_at_request',
       'requester_number_of_posts_at_request',
       'requester_number_of_posts_on_raop_at_request',
       'requester_number_of_subreddits_at_request',
       'requester_subreddits_at_request',
       'requester_upvotes_minus_downvotes_at_request',
       'requester_upvotes_plus_downvotes_at_request', 'requester_username',
       'unix_timestamp_of_request', 'unix_timestamp_of_request_utc', 'requester_received_pizza']
short_train_df = train_df[available_cols]
short_train_df.columns
no_pizza_df = short_train_df[short_train_df.requester_received_pizza == False]
yes_pizza_df = short_train_df[short_train_df.requester_received_pizza == True]

In [194]:
# stopwords from stopwords-json
stopwords_json = {"en":["a","a's","able","about","above","according","accordingly","across","actually","after","afterwards","again","against","ain't","all","allow","allows","almost","alone","along","already","also","although","always","am","among","amongst","an","and","another","any","anybody","anyhow","anyone","anything","anyway","anyways","anywhere","apart","appear","appreciate","appropriate","are","aren't","around","as","aside","ask","asking","associated","at","available","away","awfully","b","be","became","because","become","becomes","becoming","been","before","beforehand","behind","being","believe","below","beside","besides","best","better","between","beyond","both","brief","but","by","c","c'mon","c's","came","can","can't","cannot","cant","cause","causes","certain","certainly","changes","clearly","co","com","come","comes","concerning","consequently","consider","considering","contain","containing","contains","corresponding","could","couldn't","course","currently","d","definitely","described","despite","did","didn't","different","do","does","doesn't","doing","don't","done","down","downwards","during","e","each","edu","eg","eight","either","else","elsewhere","enough","entirely","especially","et","etc","even","ever","every","everybody","everyone","everything","everywhere","ex","exactly","example","except","f","far","few","fifth","first","five","followed","following","follows","for","former","formerly","forth","four","from","further","furthermore","g","get","gets","getting","given","gives","go","goes","going","gone","got","gotten","greetings","h","had","hadn't","happens","hardly","has","hasn't","have","haven't","having","he","he's","hello","help","hence","her","here","here's","hereafter","hereby","herein","hereupon","hers","herself","hi","him","himself","his","hither","hopefully","how","howbeit","however","i","i'd","i'll","i'm","i've","ie","if","ignored","immediate","in","inasmuch","inc","indeed","indicate","indicated","indicates","inner","insofar","instead","into","inward","is","isn't","it","it'd","it'll","it's","its","itself","j","just","k","keep","keeps","kept","know","known","knows","l","last","lately","later","latter","latterly","least","less","lest","let","let's","like","liked","likely","little","look","looking","looks","ltd","m","mainly","many","may","maybe","me","mean","meanwhile","merely","might","more","moreover","most","mostly","much","must","my","myself","n","name","namely","nd","near","nearly","necessary","need","needs","neither","never","nevertheless","new","next","nine","no","nobody","non","none","noone","nor","normally","not","nothing","novel","now","nowhere","o","obviously","of","off","often","oh","ok","okay","old","on","once","one","ones","only","onto","or","other","others","otherwise","ought","our","ours","ourselves","out","outside","over","overall","own","p","particular","particularly","per","perhaps","placed","please","plus","possible","presumably","probably","provides","q","que","quite","qv","r","rather","rd","re","really","reasonably","regarding","regardless","regards","relatively","respectively","right","s","said","same","saw","say","saying","says","second","secondly","see","seeing","seem","seemed","seeming","seems","seen","self","selves","sensible","sent","serious","seriously","seven","several","shall","she","should","shouldn't","since","six","so","some","somebody","somehow","someone","something","sometime","sometimes","somewhat","somewhere","soon","sorry","specified","specify","specifying","still","sub","such","sup","sure","t","t's","take","taken","tell","tends","th","than","thank","thanks","thanx","that","that's","thats","the","their","theirs","them","themselves","then","thence","there","there's","thereafter","thereby","therefore","therein","theres","thereupon","these","they","they'd","they'll","they're","they've","think","third","this","thorough","thoroughly","those","though","three","through","throughout","thru","thus","to","together","too","took","toward","towards","tried","tries","truly","try","trying","twice","two","u","un","under","unfortunately","unless","unlikely","until","unto","up","upon","us","use","used","useful","uses","using","usually","uucp","v","value","various","very","via","viz","vs","w","want","wants","was","wasn't","way","we","we'd","we'll","we're","we've","welcome","well","went","were","weren't","what","what's","whatever","when","whence","whenever","where","where's","whereafter","whereas","whereby","wherein","whereupon","wherever","whether","which","while","whither","who","who's","whoever","whole","whom","whose","why","will","willing","wish","with","within","without","won't","wonder","would","wouldn't","x","y","yes","yet","you","you'd","you'll","you're","you've","your","yours","yourself","yourselves","z","zero"]}
stopwords_json_en = set(stopwords_json['en'])
stopwords_nltk_en = set(stopwords.words('english') + ['though','pizza', 'request', 'hey', 'hi'])
stopwords_punct = set(punctuation)
# combine the stopwords.
stoplist_combined = set.union(stopwords_json_en, stopwords_nltk_en, stopwords_punct)

In [67]:
import contractions 
# import sys  
# !{sys.executable} -m pip install contractions

In [141]:
wnl = WordNetLemmatizer()

def penn2morphy(penntag):
    """ Converts Penn Treebank tags to WordNet. """
    morphy_tag = {'NN':'n', 'JJ':'a',
                  'VB':'v', 'RB':'r'}
    try:
        return morphy_tag[penntag[:2]]
    except:
        return 'n' 
    
def lemmatize_sent(text): 
    # Text input is string, returns lowercased strings.
    return [wnl.lemmatize(word.lower(), pos=penn2morphy(tag)) 
            for word, tag in pos_tag(word_tokenize(text))]

def preprocess_text(text):
    # Input: str, i.e. document/sentence
    # Output: list(str) , i.e. list of lemmas
    processed_list = [word for word in lemmatize_sent(contractions.fix(re.sub(r'http\S+', '', text)))
            if word not in stoplist_combined
            and not word.isdigit()]
    processed_text = ' '.join(processed_list) 
    return processed_text

In [145]:
short_train_df['request_text_edit_aware'][6]

"We're in Tampa Florida...moving to Ybor on Friday. My email is usfstuff@yahoo.com, since I'm having major trouble figuring out reddit's message system.\n\n\nMy two roommates and I have found a new place to live and are moving in on Friday. The rent is $1100 including utitilies and internet. My 2 roomies have very nicely offered for me to just pay $200 a month until I have a job. I am so thankful to have them as friends! This month all of us are extremely poor from paying $2000 to landlord (first+last rent+deposit). We also had to pay $310 for TECO to turn on the electricity.\n\n\nI would just really love to do something nice for them and pick them up a pizza and a sub/pzone. My one roomie doesn't love pizza and she only ever gets a sub/pzone haha. Using a few coupons, I can get 8 pizza rollers, 1 large pizza, and a pzone for $11 from Pizza Hut. If anyone can help me out I would be so grateful! Pizza Hut is just the cheapest, if you have something else in mind let me know. I have an in

In [146]:
preprocess_text(short_train_df['request_text_edit_aware'][6])

"tampa florida ... move ybor friday email usfstuff yahoo.com major trouble figure reddit 's message system roommate find place live move friday rent include utitilies internet roomy nicely offer pay month job thankful friend month extremely poor pay landlord first+last rent+deposit pay teco turn electricity love nice pick sub/pzone roomie love sub/pzone haha coupon roller large pzone hut grateful hut cheap mind interview thursday finger cross pic huge teco deposit"

In [45]:
# polarity ranges from -1 to 1, with -1 being negative and 1 being positive.
print(TextBlob(short_train_df['request_text_edit_aware'][0]).polarity)
# subjectivity which ranges from 0 to 1, with 0 being objective and 1 being subjective.
print(TextBlob(short_train_df['request_text_edit_aware'][0]).subjectivity)

0.10166666666666666
0.31333333333333335


# CountVectorizer

In [147]:
# Initialize bigram vectorizer and 
# override the analyzer totally with the preprocess_text()
count_vect = CountVectorizer(analyzer = 'word', preprocessor = preprocess_text, ngram_range = (2, 2))
short_train_df_cv = count_vect.fit_transform(short_train_df['request_text_edit_aware'])

# count frequency of ngrams
count_values = short_train_df_cv.toarray().sum(axis=0)
# list of ngrams
vocab = count_vect.vocabulary_
df_bigram = pd.DataFrame(sorted([(count_values[i],k) for k,i in vocab.items()], reverse=True)
            ).rename(columns={0: 'count', 1:'bigram'})
df_bigram['polarity'] = df_bigram['bigram'].apply(lambda x: TextBlob(x).polarity)
df_bigram['subjective'] = df_bigram['bigram'].apply(lambda x: TextBlob(x).subjectivity)

In [154]:
df_bigram[:10]

,frequency,bigram,polarity,subjective
0,471,pay forward,0.0,0.0
1,147,college student,0.0,0.0
2,142,bank account,0.0,0.0
3,129,lose job,0.0,0.0
4,128,return favor,0.0,0.0
5,121,sob story,0.0,0.0
6,94,papa john,0.0,0.0
7,92,pay back,0.0,0.0
8,89,pay rent,0.0,0.0
9,86,random act,-0.5,0.5


In [192]:
# Initialize bigram vectorizer on those who did not get pizza
count_vect = CountVectorizer(analyzer = 'word', preprocessor = preprocess_text, ngram_range = (2, 2))
short_train_df_cv = count_vect.fit_transform(no_pizza_df['request_text_edit_aware'])

# count frequency of ngrams
count_values = short_train_df_cv.toarray().sum(axis=0)
# list of ngrams
vocab = count_vect.vocabulary_
df_bigram = pd.DataFrame(sorted([(count_values[i],k) for k,i in vocab.items()], reverse=True)
            ).rename(columns={0: 'count', 1:'bigram'})
df_bigram['polarity'] = df_bigram['bigram'].apply(lambda x: TextBlob(x).polarity)
df_bigram['subjective'] = df_bigram['bigram'].apply(lambda x: TextBlob(x).subjectivity)

In [193]:
df_bigram[:10]

,frequency,bigram,polarity,subjective
0,317,pay forward,0.0,0.0
1,102,college student,0.0,0.0
2,98,bank account,0.0,0.0
3,94,return favor,0.0,0.0
4,90,sob story,0.0,0.0
5,88,lose job,0.0,0.0
6,64,random act,-0.5,0.5
7,63,pay rent,0.0,0.0
8,62,papa john,0.0,0.0
9,60,hey guy,0.0,0.0


In [185]:
# Initialize bigram vectorizer on those who did get pizza
count_vect = CountVectorizer(analyzer = 'word', preprocessor = preprocess_text, ngram_range = (2, 2))
short_train_df_cv = count_vect.fit_transform(yes_pizza_df['request_text_edit_aware'])

# count frequency of ngrams
count_values = short_train_df_cv.toarray().sum(axis=0)
# list of ngrams
vocab = count_vect.vocabulary_
df_bigram = pd.DataFrame(sorted([(count_values[i],k) for k,i in vocab.items()], reverse=True)
            ).rename(columns={0: 'count', 1:'bigram'})
df_bigram['polarity'] = df_bigram['bigram'].apply(lambda x: TextBlob(x).polarity)
df_bigram['subjective'] = df_bigram['bigram'].apply(lambda x: TextBlob(x).subjectivity)

In [190]:
df_bigram[:10]

,frequency,bigram,polarity,subjective
0,154,pay forward,0.0,0.0
1,45,college student,0.0,0.0
2,44,bank account,0.0,0.0
3,41,lose job,0.0,0.0
4,34,return favor,0.0,0.0
5,34,pay back,0.0,0.0
6,32,papa john,0.0,0.0
7,31,sob story,0.0,0.0
8,27,pay friday,0.0,0.0
9,27,find job,0.0,0.0


In [149]:
# Initialize trigram vectorizer
cv_trigram = CountVectorizer(analyzer = 'word', preprocessor = preprocess_text, ngram_range = (3, 3))
cv_trigram_fit = cv_trigram.fit_transform(short_train_df['request_text_edit_aware'])

# count frequency of ngrams
count_values = cv_trigram_fit.toarray().sum(axis=0)
# list of ngrams
trigram_vocab = cv_trigram.vocabulary_
df_trigram = pd.DataFrame(sorted([(count_values[i],k) for k,i in trigram_vocab.items()], reverse=True)
            ).rename(columns={0: 'count', 1:'trigram'})
df_trigram['polarity'] = df_trigram['trigram'].apply(lambda x: TextBlob(x).polarity)
df_trigram['subjective'] = df_trigram['trigram'].apply(lambda x: TextBlob(x).subjectivity)

In [155]:
df_trigram[df_trigram.frequency>10][:10]

,frequency,trigram,polarity,subjective
0,54,promise pay forward,0.000000,0.000
1,45,long story short,-0.025000,0.350
2,31,pay forward pay,0.000000,0.000
3,24,broke college student,0.000000,0.000
4,19,part time job,0.000000,0.000
5,18,work part time,0.000000,0.000
6,18,love pay forward,0.500000,0.600
7,18,long time lurker,-0.050000,0.400
8,17,pay forward future,0.000000,0.125
9,17,live paycheck paycheck,0.136364,0.500


In [ ]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.pipeline import make_pipeline


In [198]:
# topics for CV bigrams
cv_bigram = CountVectorizer(analyzer = 'word', preprocessor = preprocess_text, ngram_range = (2, 2))
lda = LatentDirichletAllocation(n_components=5)
pipe = make_pipeline(cv_bigram, lda)
pipe.fit(short_train_df['request_text_edit_aware'])
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += ", ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()
print_top_words(lda, cv_bigram.get_feature_names(), n_top_words=3)

Topic #0: pay forward, bank account, return favor
Topic #1: pay forward, college student, lose job
Topic #2: pay forward, bank account, pay week
Topic #3: pay forward, sob story, lose job
Topic #4: pay forward, college student, bank account



In [197]:
# topics for CV trigrams
cv_trigram = CountVectorizer(analyzer = 'word', preprocessor = preprocess_text, ngram_range = (3, 3))
lda = LatentDirichletAllocation(n_components=5)
pipe = make_pipeline(cv_trigram, lda)
pipe.fit(short_train_df['request_text_edit_aware'])
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += ", ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()
print_top_words(lda, cv_trigram.get_feature_names(), n_top_words=3)

Topic #0: promise pay forward, broke college student, make ends meet
Topic #1: long story short, promise pay forward, broke college student
Topic #2: long story short, promise pay forward, long time lurker
Topic #3: poor college student, long story short, promise pay forward
Topic #4: pay forward pay, promise pay forward, long story short



In [226]:
# Initialize bigram TDIFD vectorizer 
tf_bigram = TfidfVectorizer(analyzer = 'word', preprocessor = preprocess_text, ngram_range = (2, 2))
short_train_df_tf_bigram = tf_bigram.fit_transform(short_train_df['request_text_edit_aware'])

# average frequency of ngrams
avg_values = short_train_df_tf_bigram.toarray().sum(axis=0)
# list of ngrams
vocab = tf_bigram.vocabulary_
df_bigram = pd.DataFrame(sorted([(avg_values[i],k) for k,i in vocab.items()], reverse=True)
            ).rename(columns={0: 'sum frequency', 1:'bigram'})
df_bigram['polarity'] = df_bigram['bigram'].apply(lambda x: TextBlob(x).polarity)
df_bigram['subjective'] = df_bigram['bigram'].apply(lambda x: TextBlob(x).subjectivity)

In [227]:
df_bigram[:10]

,sum frequency,bigram,polarity,subjective
0,43.409068,pay forward,0.0,0.0
1,20.213445,college student,0.0,0.0
2,17.103988,return favor,0.0,0.0
3,16.499367,sob story,0.0,0.0
4,15.310112,bank account,0.0,0.0
5,14.238537,lose job,0.0,0.0
6,11.831305,papa john,0.0,0.0
7,11.485988,pay back,0.0,0.0
8,10.047579,couple day,0.0,0.0
9,10.012277,pay week,0.0,0.0


In [228]:
# Initialize bigram TDIFD vectorizer 
tf_trigram = TfidfVectorizer(analyzer = 'word', preprocessor = preprocess_text, ngram_range = (3, 3))
short_train_df_tf_trigram = tf_trigram.fit_transform(short_train_df['request_text_edit_aware'])

# average frequency of ngrams
num_values = short_train_df_tf_trigram.toarray().sum(axis=0)
# list of ngrams
vocab = tf_trigram.vocabulary_
df_trigram = pd.DataFrame(sorted([(num_values[i],k) for k,i in vocab.items()], reverse=True)
            ).rename(columns={0: 'num frequency', 1:'trigram'})
df_trigram['polarity'] = df_trigram['trigram'].apply(lambda x: TextBlob(x).polarity)
df_trigram['subjective'] = df_trigram['trigram'].apply(lambda x: TextBlob(x).subjectivity)

In [229]:
df_trigram[:10]

,num frequency,trigram,polarity,subjective
0,7.074477,promise pay forward,0.000,0.000
1,5.217204,broke college student,0.000,0.000
2,5.216487,long story short,-0.025,0.350
3,4.758339,pay forward pay,0.000,0.000
4,2.980866,love pay forward,0.500,0.600
5,2.972173,poor college student,-0.400,0.600
6,2.864699,pay forward future,0.000,0.125
7,2.806495,pay forward friday,0.000,0.000
8,2.780547,gladly pay forward,0.500,1.000
9,2.674388,lose job week,0.000,0.000


In [158]:
# topics for TFIDF trigrams using LDA
tfidf_vectorizer = TfidfVectorizer(analyzer = 'word', preprocessor = preprocess_text, ngram_range = (3, 3))
lda = LatentDirichletAllocation(n_components=5)
pipe = make_pipeline(tfidf_vectorizer, lda)
pipe.fit(short_train_df['request_text_edit_aware'])
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += ", ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()
print_top_words(lda, tfidf_vectorizer.get_feature_names(), n_top_words=3)

Topic #0: sob story hungry, pay forward pay, pay forward money
Topic #1: eat ramen noodle, food money kind, pepperroni mushroom visitor
Topic #2: promise pay forward, pay forward pay, hungry college student
Topic #3: broke college student, lose job couple, promise pay forward
Topic #4: promise pay forward, broke college student, happy pay forward



In [160]:
## topics for TFIDF trigrams using NML
tfidf_vectorizer = TfidfVectorizer(analyzer = 'word', preprocessor = preprocess_text, ngram_range = (3, 3))
nmf = NMF(n_components=5)
pipe = make_pipeline(tfidf_vectorizer, nmf)
pipe.fit(short_train_df['request_text_edit_aware'])
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += ", ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()
print_top_words(nmf, tfidf_vectorizer.get_feature_names(), n_top_words=3)

Topic #0: broke college student, college student pay, hey broke college
Topic #1: promise pay forward, forward pass test, pass test awww
Topic #2: pay forward pay, forward pay week, week buy pay
Topic #3: pay forward friday, happy pay forward, friday columbus ohio
Topic #4: long story short, break college student, hungry college student



NML seems to be better

In [234]:
## topics for TFIDF trigrams using NML for those who did get piza
tfidf_vectorizer = TfidfVectorizer(analyzer = 'word', preprocessor = preprocess_text, ngram_range = (3, 3))
nmf = NMF(n_components=5)
pipe = make_pipeline(tfidf_vectorizer, nmf)
pipe.fit(short_train_df.request_text_edit_aware[short_train_df.requester_received_pizza == True])
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += ", ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()
print_top_words(nmf, tfidf_vectorizer.get_feature_names(), n_top_words=3)

Topic #0: back work sound, pile bad news, careful food fridge
Topic #1: pay forward money, bean ramen pasta, rice bean ramen
Topic #2: pay forward feel, food house fund, thought give break
Topic #3: grateful pay forward, pay forward day, forever grateful pay
Topic #4: promise pay forward, life promise pay, people day life



In [235]:
## topics for TFIDF trigrams using NML for those who did not get piza
tfidf_vectorizer = TfidfVectorizer(analyzer = 'word', preprocessor = preprocess_text, ngram_range = (3, 3))
nmf = NMF(n_components=5)
pipe = make_pipeline(tfidf_vectorizer, nmf)
pipe.fit(short_train_df.request_text_edit_aware[short_train_df.requester_received_pizza == False])
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += ", ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()
print_top_words(nmf, tfidf_vectorizer.get_feature_names(), n_top_words=3)

Topic #0: broke college student, college student pay, student pay forward
Topic #1: promise pay forward, pass test awww, forward pass test
Topic #2: pay forward pay, forward pay week, week buy pay
Topic #3: sob story hungry, story hungry bore, cry sob story
Topic #4: hungry college student, student happy eat, feel donate hungry



College/student shows up more in not getting a pizza.
Being grateful seems to show up in more topics when a pizza is received.
Way to make pay if forward like phrases one topic?